In [ ]:
import strax
import straxen

In [ ]:
import wfsim

In [ ]:
import numpy as np
import pandas as pd

instruction_dtype = [('event_number', np.int), ('type', '<U2'), ('t', np.int), 
    ('x', np.float32), ('y', np.float32), ('z', np.float32), 
    ('amp', np.int), ('recoil', '<U2')]

def rand_instructions(c):
    n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
    c['total_time'] = c['chunk_size'] * c['nchunk']

    instructions = np.zeros(2 * n, dtype=instruction_dtype)
    uniform_times = c['total_time'] * (np.arange(n) + 0.5) / n
    instructions['t'] = np.repeat(uniform_times, 2) * int(1e9)
    instructions['event_number'] = np.digitize(instructions['t'], 
         1e9 * np.arange(c['nchunk']) * c['chunk_size']) - 1
    instructions['type'] = np.tile(['s1', 's2'], n)
    instructions['recoil'] = ['er' for i in range(n * 2)]

    r = np.sqrt(np.random.uniform(0, 2500, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    instructions['x'] = np.repeat(r * np.cos(t), 2)
    instructions['y'] = np.repeat(r * np.sin(t), 2)
    instructions['z'] = np.repeat(np.random.uniform(-100, 0, n), 2)

    nphotons = np.random.uniform(2000, 2050, n)
    nelectrons = 10 ** (np.random.uniform(1, 4, n))
    instructions['amp'] = np.vstack([nphotons, nelectrons]).T.flatten().astype(int)

    return instructions

c = dict(event_rate = 100, chunk_size=1, nchunk=10)
inst = rand_instructions(c)
_ = pd.DataFrame(inst).to_csv('test_uni.csv', index=False)

In [ ]:
st = strax.Context(
register=[wfsim.RawRecordsFromFax],
config=dict(fax_file='./test_uni.csv'),
**straxen.contexts.common_opts)

In [ ]:
# Just some id from post-SR1, so the corrections work
run_id = '180519_1902'

In [ ]:
# Call for event_info so it immediately get processed as well
!rm -r strax_data
st.make(run_id, 'event_info')

In [ ]:
truth = st.get_df(run_id, 'truth')
data = st.get_df(run_id, 'event_info')

In [ ]:
n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
c['total_time'] = c['chunk_size'] * c['nchunk']
timing_grid = np.linspace(0, c['total_time'], n+1) * 1e9

In [ ]:
truth['merge_index'] = np.digitize(truth['t'], timing_grid)
data['merge_index'] = np.digitize(data['time'], timing_grid)

truth.drop('event_number', axis=1, inplace=True)
data = data.merge(truth[truth['type']=='s2'], how='outer', left_on='merge_index', right_on='merge_index')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from multihist import Histdd, Hist1d

plt.rcParams['figure.figsize'] = (10, 5)
fig = plt.figure()

ax = fig.add_subplot(121)
mh = Histdd(data.s2_area, data.n_photon,
            bins=[np.logspace(1, 6, 101), np.logspace(1, 6, 101)])
plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
plt.xscale('log'); plt.yscale('log')

ax = fig.add_subplot(122)
mh = Histdd(data.n_photon, (data.s2_area-data.n_photon)/data.n_photon, 
            bins=[np.logspace(1, 6, 101), np.linspace(-0.5, 0.5, 101)])
plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
plt.xscale('log')

plt.show()

In [ ]:
truth = st.get_df(run_id, 'truth')
data = st.get_df(run_id, 'event_info')

truth['merge_index'] = np.digitize(truth['t'], timing_grid)
data['merge_index'] = np.digitize(data['time'], timing_grid)

truth.drop('event_number', axis=1, inplace=True)
data = data.merge(truth[truth['type']=='s1'], how='outer', left_on='merge_index', right_on='merge_index')

In [ ]:
plt.rcParams['figure.figsize'] = (10, 5)
fig = plt.figure()

ax = fig.add_subplot(121)
mh = Histdd(data.s1_area, data.n_photon,
            bins=[np.logspace(2, 2.8, 101), np.logspace(2, 2.8, 101)])
plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
plt.xscale('log'); plt.yscale('log')

ax = fig.add_subplot(122)
mh = Histdd(data.n_photon, (data.s1_area-data.n_photon)/data.n_photon, 
            bins=[np.logspace(2, 2.8, 101), np.linspace(-0.5, 0.5, 101)])
plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
plt.xscale('log')

plt.show()